In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
from configs import *
from image_segmenter import segment_img
import matplotlib.pyplot as plt


class_labels = ['0','1','2','3','4','5','6','7','8','9',r'\div','dot',r'\downarrow',r'\leftarrow',r'\leftrightarrow',r'\times','+',r'\rightarrow','-', r'\uparrow',r'\updownarrow','x','y','z']

mod: tf.saved_model = None
def identify_image(file) -> tuple[str, float]:
    global mod
    if mod == None:
        # mod = tf.saved_model.load('exported_model')
        mod = tf.saved_model.load('saved_model')
    image = Image.open(file).resize((224,224))
    image = np.expand_dims(np.array(image), axis=0) #adds batch dimensions and convert to numpy array
    image = image / 255.0 # Normalize pixel values to [0,1]

    #convert image to tf.Tensor
    image = tf.convert_to_tensor(image, dtype=tf.float32)

    prediction = mod(image)
    max_index = tf.argmax(prediction, axis=1)
    max_value = tf.reduce_max(prediction) 
    Mi = max_index.numpy()
    Mv = max_value.numpy()   
    max_index = tf.argmax(prediction, axis = 1)
    predict_class_index = Mi[0]
    predict_class = class_labels[predict_class_index]

    return (predict_class, Mv)

def convert_to_LaTeX(png_file_path, print_stuff=True) -> str:
    # segement
    num_parts: int = segment_img(png_file_path, "temp_dir", delete_if_exists=True, boarder_size=3)
    # identify each part
    parts_classes: list[str] = []
    for i in range(num_parts):
        png_part = f"temp_dir\\symbol_{i}.png"
        x_class, y = identify_image(png_part)
        # print(str(x_class) + " confidence: " + str(y))
        # continue
        parts_classes.append(x_class)
    # combine
    parts_classes = [x for x in parts_classes]
    if print_stuff: print(f'{" ".join(parts_classes)}')




This is a markdown cell (evil)

In [2]:
convert_to_LaTeX(r'C:\Users\walte\Documents\cs470\Final_Project\png_dataset\2033.png')

6 \leftarrow 6 \leftrightarrow dot 6


In [3]:
import csv

def csv_to_dict(csv_file) -> dict[str, str]:
    result_dict = {}
    with open(csv_file, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if len(row) == 2:
                try:
                    key = row[0]  # Assuming the first column contains integers
                    value = row[1]
                    result_dict[key] = value
                except ValueError:
                    print("Skipping row:", row, " - First column is not an integer.")
            else:
                print("Skipping row:", row, " - Expected 2 columns.")
    return result_dict


In [4]:
from cgi import test
import os
def test_accuracy():
    dataset_path: str = r'C:\Users\walte\Documents\cs470\Final_Project\archive\dataset_4 BIGGER latex equations (arrows)\png_dataset'
    key_path: str = r'C:\Users\walte\Documents\cs470\Final_Project\key.csv'
    csv_dict: dict[str, str] = csv_to_dict(key_path)
    num_success: int  =0
    num_files: int = 0
    # Iterate over files in the directory
    for filename in os.listdir(dataset_path):
        file_path = os.path.join(dataset_path, filename)
        # Check if the file is a PNG file
        if not os.path.isfile(file_path) or not filename.endswith('.png'):
            continue
        result: str = convert_to_LaTeX(file_path, print_stuff=False)
        expected: str = csv_dict[filename[:-4]]
        if result == expected: num_success += 1
        num_files += 1
    print(f"accuracy: {num_success} / {num_files} = {float(num_success)/float(num_files)}")
test_accuracy()
        

C:\Users\walte\AppData\Local\Temp\ipykernel_15936\2038173054.py:1: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  from cgi import test
